In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
DATA_PATH = Path(".") / "../../data"
RESULTS_PATH = Path(".") / "../../results"
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

In [4]:
therapeutics_ground_truth_path = DATA_PATH / "therapeutics" / "therapeutics_ground_truth_human_new_3_0.csv"
regression_results_path = RESULTS_PATH / "therapeutics" / "therapeutics_regression_results.csv"

In [5]:
import pandas as pd
# Read the CSV and save as FASTA
therapeutics_input_path = DATA_PATH / "therapeutics" / "therapeutics_gt_aa.fasta"

df = pd.read_csv(therapeutics_ground_truth_path)[['sequence_header', 'sequence_aa']]

# Save as FASTA file
with open(therapeutics_input_path, 'w') as f:
    for _, row in df.iterrows():
        f.write(f">{row['sequence_header']}\n")
        f.write(f"{row['sequence_aa']}\n")

In [6]:
from riot_na.api.api_mp import run_on_file_mp
from riot_na.api.api_mp import GENE_DB_DIR, InputType, Organism

run_on_file_mp(
    db_dir=GENE_DB_DIR,
    input_fasta_path=therapeutics_input_path,
    result_path=regression_results_path,
    input_type=InputType.AA,
    allowed_species=[Organism.HOMO_SAPIENS],
    extend_alignment=False,
)

100%|██████████| 1759/1759 [00:01<00:00, 1390.82it/s]


In [7]:
from notebooks.utils import base64_decode_series, validation_flags_comparison
import pandas as pd

new_df = pd.read_csv(regression_results_path, index_col=0, engine="pyarrow")
print(len(new_df))
new_df = base64_decode_series(new_df, "additional_validation_flags")
print(len(new_df))

old_df = pd.read_csv(therapeutics_ground_truth_path, index_col=0, engine="pyarrow") 
old_df = base64_decode_series(old_df, "additional_validation_flags")
print(len(old_df))

validation_flags_comparison(old_df, new_df)

1759
1759
1759


,False_old,False_new,False_old-new,True_old,True_new,True_old-new
regions_aa_in_aligned_sequence_aa,66.0,0.0,66.0,1692.0,1758.0,-66.0
cdr3_aa_in_junction_aa,0.0,0.0,0.0,1758.0,1758.0,0.0
locus_as_in_v_gene,0.0,0.0,0.0,1758.0,1758.0,0.0
v_gene_alignment_aa,0.0,0.0,0.0,1758.0,1758.0,0.0
j_gene_alignment_aa,0.0,0.0,0.0,1758.0,1758.0,0.0
no_negative_offsets_inside_v_alignment_aa,0.0,0.0,0.0,1758.0,1758.0,0.0
no_negative_offsets_inside_j_alignment_aa,0.0,0.0,0.0,1758.0,1758.0,0.0
consecutive_offsets_aa,0.0,0.0,0.0,1758.0,1758.0,0.0
no_empty_cdr3_aa,0.0,0.0,0.0,1758.0,1758.0,0.0
primary_sequence_in_sequence_alignment_aa,66.0,0.0,66.0,1692.0,1758.0,-66.0


In [9]:
# Merge dataframes
merged_df = old_df.merge(new_df, on='sequence_header', how='left', suffixes=('_x', '_y'))
merged_df = merged_df.fillna(False)

# Define columns to exclude from comparison
exclude_cols = {"sequence_header", "sequence_aa", "sequence_nt", "additional_validation_flags", "exc", "c_alignment_start", "c_alignment_end", "c_alignment_start_aa", "c_alignment_end_aa"}

# Get columns to compare (vectorized approach)
compare_cols = [col for col in old_df.columns if col not in exclude_cols]

if compare_cols:
    # Vectorized comparison - do all columns at once
    x_cols = [f"{col}_x" for col in compare_cols]
    y_cols = [f"{col}_y" for col in compare_cols]
    
    # Create comparison dataframe in one operation
    comparison_data = merged_df[x_cols].values == merged_df[y_cols].values
    comparison_df = pd.DataFrame(
        comparison_data,
        columns=[f"{col}_comparison" for col in compare_cols],
        index=merged_df.index
    )
    
    # Concatenate once instead of multiple times
    merged_df = pd.concat([merged_df, comparison_df], axis=1)

# Get comparison columns and compute counts
comparison_columns = [f"{col}_comparison" for col in compare_cols]
if comparison_columns:
    # More efficient counting using value_counts with normalize=False
    comparison_counts = pd.DataFrame({
        col: merged_df[col].value_counts().reindex([False, True], fill_value=0)
        for col in comparison_columns
    }).T
    
    # Display results
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(comparison_counts)

/tmp/ipykernel_263891/1245321829.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.fillna(False)


,False,True
numbering_scheme_comparison,0,1759
locus_comparison,0,1759
stop_codon_comparison,0,1759
productive_comparison,0,1759
complete_vdj_comparison,0,1759
v_call_comparison,52,1707
j_call_comparison,0,1759
c_call_comparison,0,1759
germline_alignment_aa_comparison,52,1707
sequence_alignment_aa_comparison,21,1738


In [10]:
import pandas as pd
# Read the CSV and save as FASTA
therapeutics_input_path = DATA_PATH / "therapeutics" / "therapeutics_linked_aa.fasta"

df = pd.read_csv(therapeutics_ground_truth_path)[['sequence_header', 'sequence_aa']]
df['therapeutic_name'] = df['sequence_header'].str.split('_', expand=False).str[0]
with open(therapeutics_input_path, 'w') as f:        
    for therapeutic_name, df_therapeutic in df.groupby('therapeutic_name'):
        f.write(f">{therapeutic_name}\n")
        # Join all sequence_aa for this therapeutic with the linker and write as a single sequence
        linker = "GGGGGGGGSGGGGGGGGG"
        joined_sequence = linker.join(df_therapeutic['sequence_aa'].tolist())
        f.write(f"{joined_sequence}\n")
    

In [11]:
from riot_na.api.api_mp import run_on_file_mp
from riot_na.api.api_mp import GENE_DB_DIR, InputType, Organism

linked_regression_results_path = RESULTS_PATH / "therapeutics" / "therapeutics_linked_regression_results.csv"
therapeutics_linked_input_path = DATA_PATH / "therapeutics" / "therapeutics_linked_aa.fasta"

run_on_file_mp(
    db_dir=GENE_DB_DIR,
    input_fasta_path=therapeutics_linked_input_path,
    result_path=linked_regression_results_path,
    input_type=InputType.AA,
    allowed_species=[Organism.HOMO_SAPIENS],
    extend_alignment=False,
    return_all_domains=True,
    n_processes=10
)


100%|██████████| 887/887 [00:01<00:00, 487.99it/s]


In [12]:
from riot_na.data.model import Locus
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
new_df = pd.read_csv(linked_regression_results_path, index_col=0, engine="pyarrow").reset_index()
new_df.head()
new_df['sequence_header'] = new_df['sequence_header'].str.cat(new_df['locus'].apply(lambda x: "heavy" if x == Locus.IGH else "light"), sep='_')
new_df = new_df.set_index('sequence_header')
new_df.head()

,sequence_aa,query_sequence_aa,numbering_scheme,locus,stop_codon,productive,complete_vdj,v_call,j_call,c_call,germline_alignment_aa,sequence_alignment_aa,segment_start,segment_end,v_alignment_start_aa,v_alignment_end_aa,j_alignment_start_aa,j_alignment_end_aa,v_sequence_alignment_aa,v_germline_alignment_aa,j_sequence_alignment_aa,j_germline_alignment_aa,c_sequence_alignment_aa,c_germline_alignment_aa,fwr1_aa,cdr1_aa,fwr2_aa,cdr2_aa,fwr3_aa,cdr3_aa,fwr4_aa,junction_aa,junction_aa_length,v_score_aa,j_score_aa,c_score_aa,v_cigar_aa,j_cigar_aa,c_cigar_aa,v_support_aa,j_support_aa,c_support_aa,v_identity_aa,j_identity_aa,c_identity_aa,v_sequence_start_aa,v_sequence_end_aa,j_sequence_start_aa,j_sequence_end_aa,c_sequence_start_aa,c_sequence_end_aa,v_germline_start_aa,v_germline_end_aa,j_germline_start_aa,j_germline_end_aa,c_germline_start_aa,c_germline_end_aa,fwr1_start_aa,fwr1_end_aa,cdr1_start_aa,cdr1_end_aa,fwr2_start_aa,fwr2_end_aa,cdr2_start_aa,cdr2_end_aa,fwr3_start_aa,fwr3_end_aa,cdr3_start_aa,cdr3_end_aa,fwr4_start_aa,fwr4_end_aa,sequence_aa_scheme_cigar,scheme_residue_mapping,positional_scheme_mapping,exc,additional_validation_flags
sequence_header,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abagovomab_heavy,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSSGGGGGGGG,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSSGGGGGGGGSGGGGGGGGGDIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQLLVYNAKTLAGGVSSRFSGSGSGTHFSLKIKSLQPEDFGIYYCQHHYGILPTFGGGTKLEIK,imgt,igh,False,True,True,IGHV1-46*01,IGHJ4*02,IGHE,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPSGGSTSYAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCAR------YFDYWGQGTLVTVSS,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS,1,127,1,98,105.0,119.0,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCAR,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPSGGSTSYAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCAR,WFAYWGQGTTVTVSS,YFDYWGQGTLVTVSS,GGG,GGG,QVKLQESGAELARPGASVKLSCKAS,GYTFTNYW,MQWVKQRPGQGLDWIGA,IYPGDGNT,RYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYC,ARGEGNYAWFAY,WGQGTTVTVSS,CARGEGNYAWFAYW,14,142.307057,29.721824,11.402871,98M,15M,3M,7.070990e-103,4.755582e-17,0.118256,0.653061,0.800000,1.0,1,98,105.0,119.0,120.0,122.0,1,98,1.0,15.0,123.0,125.0,1,25,26,33,34,50,51,58,59,96,97,108,109,119,9M1D20M4D25M2D11M1D37M1D17M,eyIxIjoiUSIsIjIiOiJWIiwiMyI6IksiLCI0IjoiTCIsIjUiOiJRIiwiNiI6IkUiLCI3IjoiUyIsIjgiOiJHIiwiOSI6IkEiLCIxMSI6IkUiLCIxMiI6IkwiLCIxMyI6IkEiLCIxNCI6IlIiLCIxNSI6IlAiLCIxNiI6IkciLCIxNyI6IkEiLCIxOCI6IlMiLCIxOSI6IlYiLCIyMCI6IksiLCIyMSI6IkwiLCIyMiI6IlMiLCIyMyI6IkMiLCIyNCI6IksiLCIyNSI6IkEiLCIyNiI6IlMiLCIyNyI6IkciLCIyOCI6IlkiLCIyOSI6IlQiLCIzMCI6IkYiLCIzNSI6IlQiLCIzNiI6Ik4iLCIzNyI6IlkiLCIzOCI6IlciLCIzOSI6Ik0iLCI0MCI6IlEiLCI0MSI6IlciLCI0MiI6IlYiLCI0MyI6IksiLCI0NCI6IlEiLCI0NSI6IlIiLCI0NiI6IlAiLCI0NyI6IkciLCI0OCI6IlEiLCI0OSI6IkciLCI1MCI6IkwiLCI1MSI6IkQiLCI1MiI6IlciLCI1MyI6IkkiLCI1NCI6IkciLCI1NSI6IkEiLCI1NiI6IkkiLCI1NyI6IlkiLCI1OCI6IlAiLCI1OSI6IkciLCI2MiI6IkQiLCI2MyI6IkciLCI2NCI6Ik4iLCI2NSI6IlQiLCI2NiI6IlIiLCI2NyI6IlkiLCI2OCI6IlQiLCI2OSI6IkgiLCI3MCI6IksiLCI3MSI6IkYiLCI3MiI6IksiLCI3NCI6IkciLCI3NSI6IksiLCI3NiI6IkEiLCI3NyI6IlQiLCI3OCI6IkwiLCI3OSI6IlQiLCI4MCI6IkEiLCI4MSI6IkQiLCI4MiI6IksiLCI4MyI6IlMiLCI4NCI6IlMiLCI4NSI6IlMiLCI4NiI6IlQiLCI4NyI6IkEiLCI4OCI6IlkiLCI4OSI6Ik0iLCI5MCI6IlEiLCI5MSI6IkwiLCI5MiI6IlMiLCI5MyI6IlMiLCI5NCI6IkwiLCI5NSI6IkEiLCI5NiI6IlMiLCI5NyI6IkUiLCI5OCI6IkQiLCI5OSI6IlMiLCIxMDAiOiJHIiwiMTAxIjoiViIsIjEwMiI6IlkiLCIxMDMiOiJZIiwiMTA0IjoiQyIsIjEwNSI6IkEiLCIxMDYiOiJSIiwiMTA3IjoiRyIsIjEwOCI6IkUiLCIxMDkiOiJHIiwiMTEwIjoiTiIsIjExMiI6IlkiLCIxMTMiOiJBIiwiMTE0IjoiVyIsIjExNSI6IkYiLCIxMTYiOiJBIiwiMTE3IjoiWSIsIjExOCI6IlciLCIxMTkiOiJHIiwiMTIwIjoiUSIsIjEyMSI6IkciLCIxMjIiOiJUIiwiMTIzIjoiVCIsIjEyNCI6IlYiLCIxMjUiOiJUIiwiMTI2IjoiViIsIjEyNyI6IlMiL

In [13]:
old_df = pd.read_csv(therapeutics_ground_truth_path, index_col=0, engine="pyarrow") 

In [14]:
# Merge dataframes
merged_df = old_df.merge(new_df, on='sequence_header', how='left', suffixes=('_x', '_y'))
merged_df = merged_df.fillna(False)

# Define columns to exclude from comparison
exclude_cols = {"sequence_header", "sequence_aa", "sequence_nt", "additional_validation_flags", "exc", 
"c_alignment_start", "c_alignment_end", "c_alignment_start_aa", "c_alignment_end_aa"}

# Get columns to compare (vectorized approach)
compare_cols = [col for col in old_df.columns if col not in exclude_cols]

if compare_cols:
    # Vectorized comparison - do all columns at once
    x_cols = [f"{col}_x" for col in compare_cols]
    y_cols = [f"{col}_y" for col in compare_cols]
    
    # Create comparison dataframe in one operation
    comparison_data = merged_df[x_cols].values == merged_df[y_cols].values
    comparison_df = pd.DataFrame(
        comparison_data,
        columns=[f"{col}_comparison" for col in compare_cols],
        index=merged_df.index
    )
    
    # Concatenate once instead of multiple times
    merged_df = pd.concat([merged_df, comparison_df], axis=1)

# Get comparison columns and compute counts
comparison_columns = [f"{col}_comparison" for col in compare_cols]
if comparison_columns:
    # More efficient counting using value_counts with normalize=False
    comparison_counts = pd.DataFrame({
        col: merged_df[col].value_counts().reindex([False, True], fill_value=0)
        for col in comparison_columns
    }).T
    
    # Display results
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(comparison_counts)

/tmp/ipykernel_263891/480463799.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.fillna(False)


,False,True
numbering_scheme_comparison,0,1759
locus_comparison,0,1759
stop_codon_comparison,0,1759
productive_comparison,0,1759
complete_vdj_comparison,0,1759
v_call_comparison,52,1707
j_call_comparison,0,1759
c_call_comparison,872,887
germline_alignment_aa_comparison,52,1707
sequence_alignment_aa_comparison,21,1738
